<a href="https://colab.research.google.com/github/NiluEst/DSSS_Homework9/blob/main/Ex9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install python-telegram-bot
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate
%pip install torch

In [ ]:
!python --version
!pip install --upgrade python-telegram-bot


In [4]:
import nest_asyncio
from google.colab import userdata
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters
import torch
from transformers import pipeline

In [ ]:
nest_asyncio.apply()

Telegram_API = userdata.get('Telegram_API')

generator = pipeline('text-generation', model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

Telegram_API = userdata.get('Telegram_API')

generator = pipeline('text-generation', model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")


async def generate_text(update: Update, context) -> None:
    user_message = update.message.text
    prompt = f"This is a chat between a smart chatbot who knows everything about animals and a user. The user asks: {user_message}\nChatbot:"
    generated_text = generator(prompt, max_length=300, num_return_sequences=1)[0]['generated_text']

    chatbot_reply = generated_text.split("Chatbot:")[-1].strip()
    final_reply = chatbot_reply.split("User:")[0].strip()
    await update.message.reply_text(final_reply)

async def start(update: Update, context) -> None:
    await update.message.reply_text('Hello! I am your chatbot. How can I help you?')

def main():
    application = Application.builder().token(Telegram_API).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate_text))

    application.run_polling()


if __name__ == '__main__':
    main()
